In [2]:
# import os
# os.environ["KERAS_BACKEND"] = "theano"

- https://keras.io/applications/
- https://github.com/yang-zhang/courses/blob/scratch/deeplearning1/nbs/lesson2.ipynb
- http://localhost:8887/notebooks/git/dog-breed-identification/fine_tune_2.ipynb
- https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [44]:
import math
import os
import datetime

import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Input, Lambda, Dense, Dropout, Flatten
from keras.models import Model, Sequential, load_model

from keras.utils import to_categorical
from keras.optimizers import RMSprop, SGD

from keras.applications import xception, inception_v3, imagenet_utils

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score

from secrets import KAGGLE_USER, KAGGLE_PW

In [2]:
competition_name = 'dog-breed-identification'
data_dir = '/opt/notebooks/data/' + competition_name + '/preprocessed'
batch_size = 16
target_size = (299, 299)
nb_classes = 120

In [3]:
def get_batch_data(data_dir, target_size=target_size, batch_size=batch_size):
    gen = image.ImageDataGenerator()
    batches = gen.flow_from_directory(data_dir+'/train', shuffle=False, target_size=target_size, batch_size=batch_size)
    batches_val = gen.flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)
    nb_batches = math.ceil(batches.n/batch_size)
    nb_batches_val = math.ceil(batches_val.n/batch_size)
    return batches, batches_val, nb_batches, nb_batches_val

### output of bottleneck model with preprocessed input

In [4]:
# create the base pre-trained model
base_model = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg')

In [5]:
# add preprocessing at the front
inputs = Input(shape=(299, 299, 3))
x = Lambda(inception_v3.preprocess_input)(inputs)
x = base_model(x)
mdl_preprocess_base_model = Model(inputs, x)

In [ ]:
batches, batches_val, nb_batches, nb_batches_val = get_batch_data(data_dir)

base_model_output = mdl_preprocess_base_model.predict_generator(batches, steps=nb_batches, verbose=1)
np.save(data_dir+'/results/base_model_output_{}'.format(base_model.name) , base_model_output)

base_model_output_val = model_base_model_output.predict_generator(batches_val, steps=nb_batches_val, verbose=1)
np.save(data_dir+'/results/base_model_output_val_{}'.format(base_model.name) , base_model_output_val)

In [6]:
base_model_output = np.load(data_dir+'/results/base_model_output_{}.npy'.format(base_model.name))
base_model_output_val = np.load(data_dir+'/results/base_model_output_val_{}.npy'.format(base_model.name))

### finetune - 1 dense layer

In [7]:
mdl_single_dense_on_base_model = Sequential(
    [Dense(nb_classes, 
           activation='softmax', 
           input_shape=(base_model_output.shape[1],)
          )]
)
mdl_single_dense_on_base_model.compile(optimizer=RMSprop(), 
                                       loss='categorical_crossentropy', 
                                       metrics=['accuracy'])

batches, batches_val, nb_batches, nb_batches_val = get_batch_data(data_dir)
y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

mdl_single_dense_on_base_model.fit(base_model_output, 
                                   y, 
                                   epochs=15, 
                                   batch_size=nb_batches, 
                                   validation_data=(base_model_output_val, y_val))

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Train on 8222 samples, validate on 2000 samples
Epoch 1/15
8222/8222 [==============================] - 2s 282us/step - loss: 2.8805 - acc: 0.5733 - val_loss: 1.5655 - val_acc: 0.8320
Epoch 2/15
8222/8222 [==============================] - 0s 8us/step - loss: 1.0438 - acc: 0.8676 - val_loss: 0.7721 - val_acc: 0.8750
Epoch 3/15
8222/8222 [==============================] - 0s 8us/step - loss: 0.5677 - acc: 0.8980 - val_loss: 0.5253 - val_acc: 0.8830
Epoch 4/15
8222/8222 [==============================] - 0s 8us/step - loss: 0.3981 - acc: 0.9101 - val_loss: 0.4337 - val_acc: 0.8895
Epoch 5/15
8222/8222 [==============================] - 0s 8us/step - loss: 0.3185 - acc: 0.9194 - val_loss: 0.3803 - val_acc: 0.8960
Epoch 6/15
8222/8222 [==============================] - 0s 8us/step - loss: 0.2639 - acc: 0.9308 - val_loss: 0.3686 - val_acc: 0.8845
Epoch 7/15
8222/8222 [==============================] - 0s

### finetune - 2 dense layers

In [8]:
mdl_2_dense_on_base_model = Sequential(
    [Dense(1024, activation='relu', input_shape=(base_model_output.shape[1],)),
    Dense(nb_classes, activation='softmax',)])

mdl_2_dense_on_base_model.compile(optimizer=RMSprop(),
                                  loss='categorical_crossentropy', 
                                  metrics=['accuracy'])

batches, batches_val, nb_batches, nb_batches_val = get_batch_data(data_dir)
y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

mdl_2_dense_on_base_model.fit(base_model_output, 
                                   y, 
                                   epochs=5, 
                                   batch_size=nb_batches, 
                                   validation_data=(base_model_output_val, y_val))

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Train on 8222 samples, validate on 2000 samples
Epoch 1/5
8222/8222 [==============================] - 1s 76us/step - loss: 2.0527 - acc: 0.5766 - val_loss: 0.7766 - val_acc: 0.7890
Epoch 2/5
8222/8222 [==============================] - 0s 14us/step - loss: 0.5375 - acc: 0.8494 - val_loss: 0.5982 - val_acc: 0.8360
Epoch 3/5
8222/8222 [==============================] - 0s 14us/step - loss: 0.4130 - acc: 0.8797 - val_loss: 0.4530 - val_acc: 0.8625
Epoch 4/5
8222/8222 [==============================] - 0s 14us/step - loss: 0.3557 - acc: 0.8915 - val_loss: 0.5506 - val_acc: 0.8315
Epoch 5/5
8222/8222 [==============================] - 0s 14us/step - loss: 0.3150 - acc: 0.9033 - val_loss: 0.4628 - val_acc: 0.8640


### finetune some conv layers

In [29]:
# create the base pre-trained model
base_model = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg')

# add preprocessing at the bottom
inputs = Input(shape=(299, 299, 3))
x = Lambda(inception_v3.preprocess_input)(inputs)
x = base_model(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(120, activation='softmax')(x)
# this is the model we will train
model = Model(inputs, predictions)

In [30]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [31]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [32]:
gen = image.ImageDataGenerator()
batches = gen.flow_from_directory(data_dir+'/train', shuffle=True, target_size=target_size, batch_size=batch_size)
batches_val = gen.flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)

nb_batches = math.ceil(batches.n/batch_size)
nb_batches_val = math.ceil(batches_val.n/batch_size)

y_encode = batches.classes
y_val_encode = batches_val.classes

y = to_categorical(batches.classes)
y_val = to_categorical(batches_val.classes)

Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.


In [33]:
model.fit_generator(batches, 
                    steps_per_epoch=nb_batches, 
                    epochs=3,
                    validation_data=batches_val,
                    validation_steps=nb_batches_val)

Epoch 1/3
514/514 [==============================] - 102s 199ms/step - loss: 1.9856 - acc: 0.5195 - val_loss: 0.8978 - val_acc: 0.7330
Epoch 2/3
514/514 [==============================] - 99s 193ms/step - loss: 1.0027 - acc: 0.7127 - val_loss: 0.8710 - val_acc: 0.7660
Epoch 3/3
514/514 [==============================] - 99s 193ms/step - loss: 0.8925 - acc: 0.7453 - val_loss: 0.7957 - val_acc: 0.7815


In [34]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_9
1 conv2d_377
2 batch_normalization_377
3 activation_377
4 conv2d_378
5 batch_normalization_378
6 activation_378
7 conv2d_379
8 batch_normalization_379
9 activation_379
10 max_pooling2d_17
11 conv2d_380
12 batch_normalization_380
13 activation_380
14 conv2d_381
15 batch_normalization_381
16 activation_381
17 max_pooling2d_18
18 conv2d_385
19 batch_normalization_385
20 activation_385
21 conv2d_383
22 conv2d_386
23 batch_normalization_383
24 batch_normalization_386
25 activation_383
26 activation_386
27 average_pooling2d_37
28 conv2d_382
29 conv2d_384
30 conv2d_387
31 conv2d_388
32 batch_normalization_382
33 batch_normalization_384
34 batch_normalization_387
35 batch_normalization_388
36 activation_382
37 activation_384
38 activation_387
39 activation_388
40 mixed0
41 conv2d_392
42 batch_normalization_392
43 activation_392
44 conv2d_390
45 conv2d_393
46 batch_normalization_390
47 batch_normalization_393
48 activation_390
49 activation_393
50 average_pooling2d_38
51 conv2d_389
52

In [35]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in base_model.layers[:249]:
   layer.trainable = False
for layer in base_model.layers[249:]:
   layer.trainable = True

In [36]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
no_of_epochs = 50
for epoch in range(no_of_epochs):
    print ("Running epoch: %d" % epoch)
    batches = gen.flow_from_directory(data_dir+'/train', shuffle=True, target_size=target_size, batch_size=batch_size)
    batches_val = gen.flow_from_directory(data_dir+'/valid', shuffle=False, target_size=target_size, batch_size=batch_size)
    model.fit_generator(batches, 
                    steps_per_epoch=nb_batches, 
                    epochs=1,
                    validation_data=batches_val,
                    validation_steps=nb_batches_val
                   )
    latest_filename = data_dir+'/results/ft_top_%d_%s.h5' %  (epoch,
        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
    )
    print(latest_filename)
    model.save(latest_filename)

Running epoch: 0
Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Epoch 1/1
514/514 [==============================] - 119s 232ms/step - loss: 0.5236 - acc: 0.8357 - val_loss: 0.6446 - val_acc: 0.8075
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_0_2017-11-16-14-46.h5
Running epoch: 1
Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Epoch 1/1
514/514 [==============================] - 115s 224ms/step - loss: 0.3941 - acc: 0.8781 - val_loss: 0.6264 - val_acc: 0.8080
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_1_2017-11-16-14-48.h5
Running epoch: 2
Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Epoch 1/1
514/514 [==============================] - 115s 224ms/step - loss: 0.3463 - acc: 0.8930 - val_loss: 0.6258 - val_acc: 0.8120
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_2_2017-11-16-14-5

514/514 [==============================] - 115s 224ms/step - loss: 0.0749 - acc: 0.9866 - val_loss: 0.6336 - val_acc: 0.8270
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_24_2017-11-16-15-33.h5
Running epoch: 25
Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Epoch 1/1
514/514 [==============================] - 115s 224ms/step - loss: 0.0671 - acc: 0.9889 - val_loss: 0.6373 - val_acc: 0.8265
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_25_2017-11-16-15-35.h5
Running epoch: 26
Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.
Epoch 1/1
514/514 [==============================] - 115s 224ms/step - loss: 0.0651 - acc: 0.9882 - val_loss: 0.6449 - val_acc: 0.8260
/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_26_2017-11-16-15-37.h5
Running epoch: 27
Found 8222 images belonging to 120 classes.
Found 2000 images belonging to 120 classes.

KeyboardInterrupt: 

In [40]:
ls /opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_5_2017-11-16-14-56.h5

/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_5_2017-11-16-14-56.h5


In [46]:
load_model??

In [45]:
model = load_model('/opt/notebooks/data/dog-breed-identification/preprocessed/results/ft_top_5_2017-11-16-14-56.h5')

NameError: name 'imagenet_utils' is not defined

### predict

In [47]:
batches_test = gen.flow_from_directory(data_dir+'/test', shuffle=False, target_size=target_size, batch_size=batch_size)

Found 10357 images belonging to 1 classes.


In [48]:
nb_batches_test = math.ceil(batches_test.n/batch_size)

In [49]:
pred = model.predict_generator(batches_test, steps=nb_batches_test, verbose=1)

648/648 [==============================] - 117s 180ms/step


In [50]:
test_ids = [f.split('/')[1].split('.')[0] for f in batches_test.filenames]

In [51]:
subm=pd.DataFrame(np.hstack([np.array(test_ids).reshape(-1, 1), pred]))
labels = pd.read_csv(data_dir+'/labels.csv')
cols = ['id']+sorted(labels.breed.unique())
subm.columns = cols

In [52]:
description = 'inception_data_finetune_more_layers'
submission_file_name = data_dir+'/results/%s_%s.csv' % (description,
                                                        datetime.datetime.now().strftime('%Y-%m-%d-%H-%M')
                                                       )
subm.to_csv(submission_file_name, index=False)

### submit

In [53]:
!kg config -u $KAGGLE_USER -p $KAGGLE_PW -c $competition_name

In [54]:
!kg submit $submission_file_name -m $description

0.65763
